<a href="https://colab.research.google.com/github/Shadycoder-rgb/Fraud-Detection/blob/main/Fraud_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
import numpy as np
import pandas as pd
data=pd.read_csv('/content/drive/MyDrive/PS_20174392719_1491204439457_log.csv')


In [12]:
data=data.drop(data.index[1000000:6362619])
data = data.drop(columns=['nameOrig', 'nameDest'], errors='ignore')
data['type'] = data['type'].astype('category').cat.codes
print("\nCorrelation with isFraud:")
print(data.corr()['isFraud'])
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
data[['amount', 'oldbalanceOrg', 'newbalanceOrig', 'oldbalanceDest', 'newbalanceDest']] = scaler.fit_transform(
    data[['amount', 'oldbalanceOrg', 'newbalanceOrig', 'oldbalanceDest', 'newbalanceDest']])
print("\nClass Distribution:\n", data['isFraud'].value_counts())



Correlation with isFraud:
step             -0.002791
type              0.013151
amount            0.075455
oldbalanceOrg     0.001089
newbalanceOrig   -0.006663
oldbalanceDest   -0.004867
newbalanceDest   -0.000577
isFraud           1.000000
isFlaggedFraud         NaN
Name: isFraud, dtype: float64

Class Distribution:
 isFraud
0    999465
1       535
Name: count, dtype: int64


In [13]:
from imblearn.over_sampling import SMOTE

X = data.drop(columns=['isFraud'])
y = data['isFraud']
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

print("\nClass Distribution After SMOTE:\n", y_resampled.value_counts())



Class Distribution After SMOTE:
 isFraud
0    999465
1    999465
Name: count, dtype: int64


In [14]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score

X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train, y_train)

y_pred = rf_model.predict(X_test)
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("ROC-AUC Score:", roc_auc_score(y_test, rf_model.predict_proba(X_test)[:, 1]))



Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00    199747
           1       1.00      1.00      1.00    200039

    accuracy                           1.00    399786
   macro avg       1.00      1.00      1.00    399786
weighted avg       1.00      1.00      1.00    399786

ROC-AUC Score: 0.9999854644558822


In [15]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier

models = {
    "Random Forest": RandomForestClassifier(random_state=42),
    "Logistic Regression": LogisticRegression(max_iter=1000),
    "Decision Tree": DecisionTreeClassifier(random_state=42),
    "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
}

for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print(f"\n{name} Results:")
    print(classification_report(y_test, y_pred))
    print("ROC-AUC Score:", roc_auc_score(y_test, model.predict_proba(X_test)[:, 1]))



Random Forest Results:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00    199747
           1       1.00      1.00      1.00    200039

    accuracy                           1.00    399786
   macro avg       1.00      1.00      1.00    399786
weighted avg       1.00      1.00      1.00    399786

ROC-AUC Score: 0.9999854644558822

Logistic Regression Results:
              precision    recall  f1-score   support

           0       0.80      0.96      0.87    199747
           1       0.95      0.75      0.84    200039

    accuracy                           0.86    399786
   macro avg       0.87      0.86      0.85    399786
weighted avg       0.87      0.86      0.85    399786

ROC-AUC Score: 0.9285806642934293

Decision Tree Results:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00    199747
           1       1.00      1.00      1.00    200039

    accuracy                        

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [06:45:40] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)



XGBoost Results:
              precision    recall  f1-score   support

           0       1.00      0.99      0.99    199747
           1       0.99      1.00      0.99    200039

    accuracy                           0.99    399786
   macro avg       0.99      0.99      0.99    399786
weighted avg       0.99      0.99      0.99    399786

ROC-AUC Score: 0.999767002472671
